# Fine Tuning

Se revisa cuál es la versión disponible de GPU para descargar las dependencias compatibles. T4 usa la versión 7, por lo que se usa la segunda instalación.

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

El modelo usado es Llama 3, y se va a usar el modelo proveniente de Unsloth que es más compacto y reducido que el modelo de Meta.


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True #Se activa con True para ahorar espacio.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", #Aquí puede ir el nombre
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.




---



En esta sección se definen cuáles son los parámetros del modelo que se van a ajustar. El Fine Tuning se realizará por medio de un LoRA (Low-Rank Adaptation), los LoRA funcionan como pequeños modelos que agregan matrices menores a la matriz del modelo original.

In [ ]:
model = FastLanguageModel.get_peft_model( #Adaptación con la técnica PEFT (Param.-Efficient Fine-Tuning), ayuda a consumir menos memoria
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], #Parámetros del modelo que se van a ajustar
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Dataset de entrenamiento

Se creó un pequeño dataset para el Fine Tuning con 234 entradas con su respuesta esperada. Para esta sección, **asegúrese se haber cargado el archivo dataset.json al entorno**.

Se crean varios prompts con base en el dataset y se guardan siguiendo la estructura del `alpaca_prompt`. Ninguna entrada del dataset contiene contexto adicional (Input).


In [ ]:
# this is basically the system prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # do not forget this part!
def formatting_prompts_func(examples):
    instructions = examples["question"]
    outputs      = examples["answer"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, "", output) + EOS_TOKEN # without this token generation goes on forever!
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files="./dataset.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

### Entrenamiento del modelo

Esta es la etapa donde se realiza el Fine Tuning propiamente. A continuación se establece el `trainer` que es que que lleva a cabo este proceso. El `trainer` recibe el modelo que se quiere ajustar, el tokenizador del modelo, el dataset de entrenamiento y el tipo de datos que contiene el dataset.

Con esta configuración, el `trainer` va a realizar 60 pasos de entrenamiento con una tasa de aprendizaje de 0.0002. Para un entrenamiento más robusto y completo, se puede cambiar `max_steps` por `None` y descomentar `num_train_epochs=1`

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Cantidad de pasos de entrenamiento, no se usa todo el dataset
        #num_train_epochs=1 #Cantidad de veces que realiza el entrenamiento con todoel datset
        learning_rate = 2e-4, #Tasa de aprendizaje, cuando menor sea, mejor
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(), #Formato de punto flotante si bfloat16 está disponible
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was c

Map (num_proc=2):   0%|          | 0/234 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Ahora, con el `trainer` completo se realiza el entrenamiento. Durante el proceso se muestra el Training Loss de cada paso de aprendizaje. El objetivo es minimizar el Training Loss, así que esperamos que llegue tan cerca de cero como sea posible.
Además, con la tasa de aprendizaje seleccionada, el Training Loss debería converger a un valor. Con los parámetros, el proceso de training tarda en torno a 5-6 minutos.

In [ ]:
#El Fine Tuning
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 234 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.300600
2,2.323300
3,2.294300
4,2.062600
5,1.869700
6,1.505000
7,1.228400
8,0.991500
9,0.801600
10,0.737200


En la mayoría de pruebas el Training Loss inicia con un valor entorno a 2.6 en su primer paso, y para el paso 60 el Training Loss se reduce a aproximandamente una décima del valor inicial. Esto demuestra que el modelo está aprendiendo del dataset, y el valor tampoco es tan pequeño para que exista un riesgo importante de sobreajuste.

### Chat Bot

Aunque la interfaz de usuario no es la mejor en el entorno de Colab, en esta sección se implementa un Chat Bot con el modelo recientemente entrenado y que ahora usa documentos para complementar sus respuestas.

Primero se instalan las dependencias necesarias.

In [ ]:
%%capture
!pip install fastembed langchain_community pdfplumber chromadb

Ahora se definen algunas constantes como la carpeta para guardar la base de datos, el objeto que permite crear los embeddings, el splitter y el prompt semilla o guía para el modelo.

In [ ]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

folder_path = "DocumentsDB"

embedding = FastEmbedEmbeddings()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024, chunk_overlap=80, length_function=len, is_separator_regex=False
)

raw_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Ahora se declaran las funciones necesarias para el Chat Bot.

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import Chroma
from transformers import TextStreamer

import re

def generate_response(query, context): #Usa el modelo y el prompt para generar la respuesta
    prompt = raw_prompt.format(query, context, "")

    #print(prompt)

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    outputs = model.generate(**inputs, max_new_tokens=128)
    decode_outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decode_outputs

def extract_response(full_response): #Limpia el prompt generado para solo mostrar la respuesta
    # Encuentra la posición de "### Response:"
    response_marker = "### Response:"
    start_index = full_response.find(response_marker)

    # Extrae la parte de la respuesta
    response_part = full_response[start_index + len(response_marker):].strip()

    # Elimina la cadena específica
    response_part = response_part.replace("<|end_of_text|>", "").strip()

    return response_part

def ask_chatbot(query): #Función principal del chatbot que recibe el query y recupera el contexto
    vector_store = Chroma(persist_directory=folder_path, embedding_function=embedding)

    retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={
            "k": 10,
            "score_threshold": 0.2,
        },
    )

    # Recuperar documentos relevantes
    docs = retriever.get_relevant_documents(query)
    context = " ".join([doc.page_content for doc in docs])

    #sources = [{"source": doc.metadata.get("source", "unknown"), "page_content": doc.page_content} for doc in docs]
    #print(f"sources: {sources}")

    # Generar respuesta usando el modelo entrenado
    response = generate_response(query, context)

    return extract_response(response)

def upload_documents(filepaths): #Sube una lista de documentos a Chroma
    all_docs = []
    for file_path in filepaths:
        loader = PDFPlumberLoader(file_path)
        docs = loader.load_and_split()
        all_docs.extend(docs)

    chunks = text_splitter.split_documents(all_docs)
    print(f"Total chunks len={len(chunks)}")

    vector_store = Chroma.from_documents(
        documents=chunks, embedding=embedding, persist_directory=folder_path
    )

    vector_store.persist()

    response = {
        "status": "Successfully Uploaded",
        "file_count": len(filepaths),
        "total_doc_len": len(all_docs),
        "chunks": len(chunks),
    }
    return response

Se suben los documentos para que sean procesados y guardados en la base de datos.

In [ ]:
# Se recomienda encarecidamente no subir más de dos documentos porque consume demasiada memoria RAM.
upload_documents(["./constitucion.pdf", "./LEY-DE-TRÁNSITO-POR-VÍAS-PÚBLICAS-9078-2022-2-236.pdf"])

Total chunks len=658


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


{'status': 'Successfully Uploaded',
 'file_count': 2,
 'total_doc_len': 284,
 'chunks': 658}

Finalmente, esta es la interfaz del chatbot. Las entradas del usuario se marcan con ">>" y las respuestas se marcan con el nombre de "ASK", el nombre que le dimos a este pequeño chatbot.

Para terminar normalmente el chatbot, puede ingresar "chao!"

In [ ]:
#Interfaz del Chatbot

user_input = ''

while (True):
  user_input = input(">> ")
  if user_input == 'chao!':
    break
  else:
    print("\nASK: ", ask_chatbot(user_input), "\n\n")

In [ ]:
ask_chatbot("¿Cuáles son los tipos de licencia de conducir?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Los tipos de licencia de conducir son: clase A para motocicletas y bicimoto, clase B para vehículos de pasajeros, clase C para vehículos de transporte público, clase D para tractores y maquinaria pesada, y clase E para vehículos de más de cuatro ejes.'

In [ ]:
ask_chatbot("¿Qué dice el artículo 5 de la Constitución sobre el territorio nacional?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'El artículo 5 de la Constitución establece que el territorio nacional está comprendido entre el mar Caribe, el Océano Pacífico y las Repúblicas de Nicaragua y Panamá, y que el Estado ejerce la soberanía completa y exclusiva en el espacio aéreo de su territorio, en sus aguas territoriales, en su plataforma continental y en su zócalo insular.'

In [ ]:
ask_chatbot("¿Qué necesita quien quiera solicitar la naturalización?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Para solicitar la naturalización en Costa Rica, el solicitante debe acreditar su buena conducta, demostrar que tiene oficio o medio de vivir conocido, y saber hablar, escribir y leer el idioma español.'

In [ ]:
ask_chatbot("¿Qué se requiere para ser Presidente de la República?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Para ser Presidente de la República se requiere ser costarricense por nacimiento o por naturalización, haber cumplido 35 años de edad, estar en pleno goce de derechos civiles y políticos, y haber residido en el país por lo menos 15 años inmediatamente anteriores a la elección.'

In [ ]:
ask_chatbot("¿Qué es una licencia tipo A1?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'La licencia tipo A1 es para motocicletas y bicicletas de menos de 125 cc y se otorga a mayores de 16 años.'

In [ ]:
ask_chatbot("¿Entre qué horas se prohíbe la circulación de los vehículos sin las luces encendidas?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'¿Qué horas se prohíbe la circulación de vehículos sin las luces encendidas?\nLa circulación de vehículos sin las luces encendidas está prohibida entre las 6:00 p.m. y las 6:00 a.m., y en condiciones climáticas que dificulten la visibilidad.'

In [ ]:
ask_chatbot("¿Cuál es la velocidad máxima permitida en una rotonda?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'La velocidad máxima permitida en una rotonda es de 30 km/h.'

In [ ]:
ask_chatbot("¿Cuál es la velocidad máxima permitida en una rotonda?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'La velocidad máxima permitida en una rotonda es de aproximadamente 20 km/h, para permitir una maniobra segura.'

In [ ]:
ask_chatbot("¿Cuál es la distancia permitida para circular en retroceso?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'La distancia permitida para circular en retroceso es de aproximadamente 50 metros, siempre y cuando se tome la debida precaución.'

In [ ]:
ask_chatbot("Con base en la Ley de Tránsito ¿Qué es una Infracción de Tránsito?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'¿Qué es una Infracción de Tránsito? Una infracción de tránsito es cualquier acción o omisión que viola las leyes y reglamentos de tránsito y puede incluir conductas como exceder el límite de velocidad, no respetar las señales de tránsito, no usar el cinturón de seguridad, etc.'

### Guardar el modelo
Si se desea, se puede guardar el LoRA del modelo. Se guarda el safetensors y el config correspondiente, y se pueden agregar a cualquier modelo compatible.

In [ ]:
model.save_pretrained("leyes_lora_model") # Se guarda en la carpeta "leyes_lora_model"